# GCT-PLS 在域自适应中的应用
## Dr. Ramin Nikzad-Langerodi
### Bottleneck Analytics GmbH
info@bottleneck-analytics.com

___
首先，我们加载一些将要使用的模块，包括 GCT-PLS 类。

In [ ]:
!pip install pandas; seaborn
import numpy as np
import pandas as pd
import seaborn as sb
import scipy.io as sio
import matplotlib.pyplot as plt

from diPLSlib.models import GCTPLS as gctpls
from diPLSlib.functions import transfer_laplacian
from diPLSlib.utils import misc as fct

# 配置中文绑图支持
fct.setup_chinese_plot()

### 数据集

我们将使用来自 Eigenvector Research Inc. 的玉米 (Corn) 数据集。该数据集包含来自两台不同仪器的玉米样品的 NIR 光谱。该数据集可在 https://www.eigenvector.com/data/Corn/ 获得。该数据集也可在本仓库的 `data` 文件夹中找到。

In [ ]:
data = sio.loadmat('../data/corn.mat')

# 光谱
m5 = data['m5spec'][0][0]['data']
mp5 = data['mp5spec'][0][0]['data']
mp6 = data['mp6spec'][0][0]['data']
wn = data['m5spec'][0][0][9][1][0][0]

SPEC = {'m5': m5, 'mp5': mp5, 'mp6': mp6}

# 属性
Y = data['propvals']['data'][0][0]
y = np.expand_dims(Y[:,0],1) # 0: 水分, 1: 油, 2: 蛋白质, 3: 淀粉

我们将 m5 和 mp6 光谱分割为训练集和测试集。我们将使用 m5 光谱及其水分含量进行校准，使用 m5 和 mp6 仪器的 NIST 标准样品进行仪器标准化，并使用 mp6 测试集进行评估。

In [ ]:
idx_train = np.random.choice(np.arange(0,80,1), 60, replace=False)
idx_test = np.delete(np.arange(0,80,1), idx_train)

m5_train = m5[idx_train,:]
m5_test = m5[idx_test,:]
mp6_test  = mp6[idx_test,:]
y_train = y[idx_train] 
y_test = y[idx_test]

# 转移标准
m5nbs = data['m5nbs'][0][0]['data']
mp5nbs = data['mp5nbs'][0][0]['data']
mp6nbs = data['mp6nbs'][0][0]['data']
mp5nbs = np.delete(mp5nbs,0,0)
mp6nbs = np.delete(mp6nbs,0,0)

NBS = {'m5nbs': m5nbs, 'mp5nbs': mp5nbs, 'mp6nbs': mp6nbs}

d = pd.DataFrame({'wn': wn, 'm5': np.mean(m5,0), 'mp6': np.mean(mp6,0), 'm5_NIST': np.mean(m5nbs,0), 'mp6_NIST': np.mean(mp6nbs,0)})

让我们来看看来自两台仪器的玉米样品和 NIST 转移标准的平均光谱。

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(121)
dd = pd.melt(d, id_vars='wn', value_vars=['m5','mp6'], var_name='光谱', value_name='吸光度')
ax = plt.gca()
sb.lineplot(x='wn', y='吸光度', hue='光谱', data=dd, ax=ax).set(title='玉米光谱')
ax.set_xlabel('波长 (nm)')

plt.subplot(122)
dd = pd.melt(d, id_vars='wn', value_vars=['m5_NIST','mp6_NIST'], var_name='光谱', value_name='吸光度')
ax = plt.gca()
sb.lineplot(x='wn', y='吸光度', hue='光谱', data=dd, ax=ax).set(title='NIST 标准')
ax.set_xlabel('波长 (nm)')

plt.tight_layout()

可以看出，来自两台仪器的玉米样品的平均光谱存在明显的偏移。与玉米样品相比，NIST 标准的光谱相当不同。

### 基于图的校正转移偏最小二乘回归 (GCT-PLS)
使用 GCT-PLS，我们将尝试结合 m5 玉米光谱以及 m5 和 mp6 NIST 标准，寻找一个校准模型，该模型可用于预测来自 mp6 仪器的玉米样品的水分含量。我们首先探索正则化如何影响校准和转移标准样品的投影。

In [ ]:
ncomp = 3
l = np.array([0, 10, 100])

In [ ]:
k = 1
plt.figure(figsize=(14, 8))
for i in l:

    m = gctpls(A=ncomp, l=i)
    m.fit(m5_train, y_train, m5nbs, mp6nbs)

    # 投影
    sub = "2"+"3"+str(k)
    plt.subplot(int(sub))
    sp = plt.scatter(m.T_[:,0], m.T_[:,1], color='k', edgecolors='k',alpha=0.25)
    plt.scatter(m.Ts_[:,0], m.Ts_[:,1], edgecolors='k',alpha=0.75)
    plt.scatter(m.Tt_[:,0], m.Tt_[:,1], edgecolors='k',alpha=0.75)   
    plt.legend(['m5 (校准)','m5-NIST','mp6-NIST'])
    el = fct.hellipse(m.T_)
    plt.plot(el[0, :], el[1, :], '-.k')
    plt.axhline(y=0,linestyle='-.', color='k')
    plt.axvline(x=0,linestyle='-.', color='k')
    plt.xlabel('LV 1')
    plt.ylabel('LV 2')
    title = '$\gamma=$' + str(i)
    plt.title(title)
    
    # 重建
    sub = "2"+"3"+str(k+3)
    plt.subplot(int(sub))
    m5_hat = m.T_@m.P_.T + m.mu_
    plt.plot(wn,np.mean(m5_hat, 0))
    mp6t = mp6_test[...,:] - m.mu_
    mp6_hat = mp6t@(m.W_@np.linalg.inv(m.P_.T@m.W_))@m.P_.T + m.mu_
    plt.plot(wn, np.mean(mp6_hat, 0))
    plt.xlabel('波长 (nm)')
    plt.ylabel('吸光度 (a.u.)')
    plt.legend(['m5 (校准)', 'mp6 (验证)'])
    k += 1

plt.tight_layout()

在图中我们可以看到，随着正则化的增加，（配对的）转移标准样品的投影变得更加接近（上图）。当从相应的潜变量重建 m5 训练集和 mp6 验证集样品时，我们看到样品的（平均）光谱变得更加相似（下图）。我们现在将比较带正则化和不带正则化的校准表现。

### 标准 PLS 模型

In [ ]:
ncomp = 3
L = transfer_laplacian(m5nbs, mp6nbs)
m_pls = gctpls(A=ncomp, l=0, rescale='Source')
m_pls.fit(m5_train, y_train, m5nbs, mp6nbs)

yhat_pls = m_pls.predict(mp6_test)

error_pls = fct.rmse(y_test, yhat_pls)

print('RMSEP (PLS): ' + str(np.round(error_pls, 2)))

### 带正则化的 GCT-PLS 模型

In [ ]:
ncomp = 3
m_gct = gctpls(A=ncomp, l=100, rescale='Source')
m_gct.fit(m5_train, y_train, m5nbs, mp6nbs)

yhat_gct = m_gct.predict(mp6_test)

error_gct = fct.rmse(y_test, yhat_gct)

print('RMSEP (GCT-PLS): ' + str(np.round(error_gct, 2)))

通过在校准模型中包含转移标准并应用适当的正则化，我们可以显著降低 mp6 样品的预测误差。